In [1]:
import MySQLdb
import pandas as pd
import numpy as np
from datetime import datetime

In [5]:
hc_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col=0)
hc_str = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col=0)

In [8]:
hc_ids = hc_se.index.tolist()
print len(hc_ids)

100


In [2]:
#접속하기
db_handle = MySQLdb.connect('localhost', 'root','comsysbio', 'COCONUTv5')
#cur = db_handle.cursor(MySQLdb.cursors.DictCursor) # query를 통해 데이터를 얻을 때 python의 dict로 얻기 위한 방법
cursor = db_handle.cursor()

In [15]:
"{}".format(tuple(hc_ids))

'(5525, 1020, 732, 9210, 926, 4020, 86025, 70460, 82835, 79944, 79942, 79938, 79941, 1530, 1531, 79939, 153, 8019, 4286, 70319, 8885, 3761, 119, 139, 70410, 70661, 4521, 1878, 622, 543, 9471, 9926, 85183, 3005, 10195, 7874, 59654, 50263, 80081, 80082, 80097, 80164, 2969, 76198, 58454, 61885, 8739, 562, 77956, 614, 40553, 773, 65817, 71038, 70458, 5529, 1166, 71664, 8024, 837, 416, 79982, 7475, 83229, 89181, 760, 10086, 31743, 2682, 823, 79684, 845, 752, 70473, 7859, 78876, 1001, 666, 1574, 58258, 7996, 1167, 71205, 43066, 9670, 141, 243, 1331, 53060, 70973, 8012, 61516, 70924, 77084, 7638, 70692, 71009, 85310, 1343, 1529)'

In [18]:
# herbal compound와 관련있는 gene id 뽑기
query1 = "SELECT compID, geneID, reference FROM Compound_Gene where compID in {} and (organism=\"human\" or organism=\"homo sapiens\")".format(tuple(hc_ids))
cursor.execute(query1); #쿼리 날리는 거
results = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [41]:
np_results = np.array(results)

In [43]:
df_results = pd.DataFrame({'compID' : np_results[:, 0], 'geneID' : np_results[:, 1], 'reference' : np_results[:, 2]})

In [122]:
print len(df_results)
print len(df_results.drop_duplicates('compID')), len(df_results.drop_duplicates('geneID'))
df_results = df_results[['compID', 'geneID']].drop_duplicates()
print len(df_results)
df_results.head(1)

35808
66 8342
18525


,compID,geneID
0,119,283


In [124]:
# gene 정보가 없는 compound들
a = [int(x) for x in df_results['compID'].drop_duplicates().tolist()]
hc_se.index[~hc_se.index.isin(a)]

Int64Index([70460, 82835, 79944, 79941, 70319,  8885, 70410, 70661, 85183,
            59654, 50263, 80081, 76198, 58454, 61885, 40553, 65817, 71038,
            70458, 71664, 89181, 31743, 79684, 70473, 71205, 53060, 70973,
            61516, 70924, 77084,  7638, 70692, 71009, 85310],
           dtype='int64')

In [125]:
hc_ids2 = hc_se.index[hc_se.index.isin(a)].tolist()

In [90]:
# geneId에 해당하는 entrezID와 uniprotID 뽑기
gene_ids = [int(x) for x in df_results.geneID.drop_duplicates().tolist()]
query2 = "SELECT geneID, entrezID, uniprotID FROM Gene where geneID in {}".format(tuple(gene_ids))
cursor.execute(query2); #쿼리 날리는 거
results2 = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [ ]:
np_results2 = np.array(results2)
df_results2 = pd.DataFrame({'geneID' : np_results2[:,0], 'entrezID' : np_results2[:,1], 'uniprotID':np_results2[:,2]})

print len(df_results2)
print len(df_results2.drop_duplicates('entrezID')), len(df_results2.drop_duplicates('geneID')) # entrez는 null 인것이 사라짐
print len(df_results2)
df_results2.head(1)

In [200]:
print len(df_results2)
print len(df_results2.drop_duplicates('entrezID')), len(df_results2.drop_duplicates('geneID')) # entrez는 null 인것이 사라짐
print len(df_results2)
df_results2.head(1)

8342
8325 8342
8342


,entrezID,geneID,uniprotID
0,2147,1,P00734 / A0N064|A8IM47|C8BKD1|E7FAN5|G1NEM6|G1...


In [239]:
# uniprot ID를 row당 하나 오게 바꾸기
aa = df_results2['uniprotID'].str.split('/').apply(pd.Series, 1).stack()
aa.index = aa.index.droplevel(-1)
aa.name = 'uniprotID'
temp = df_results2.copy()
del temp['uniprotID']
bb = temp.join(aa)
cc = bb['uniprotID'].str.split('|').apply(pd.Series, 1).stack()
cc.index = cc.index.droplevel(-1)
cc.name = 'uniprotID'
temp = bb.copy()
del temp['uniprotID']
dd = temp.join(cc)

In [271]:
df_results3 = dd.drop_duplicates()

In [426]:
df_results3.head(1)

,entrezID,geneID,uniprotID
0,2147,1,P00734


In [272]:
merged = pd.merge(left = df_results, right = df_results3, how='inner')

In [429]:
print len(df_results), len(df_results3)
print len(merged), len(merged.drop_duplicates())
print len(merged.drop_duplicates('compID')), len(merged.drop_duplicates('entrezID')), len(merged.drop_duplicates('uniprotID'))
merged.head(1)

18525 236703
595543 595543
66 8325 236321


,compID,geneID,entrezID,uniprotID
0,119,283,4846,P29474


In [344]:
# herbal compounds의 gene id를 uniprot에서 입력으로 넣어서 가져온 protein 정보
hc_seq_raw = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/hc_seq_raw.tab")

In [430]:
hc_seq_raw.head(1)

,Entry,Entry name,Sequence,Length,Organism ID
0,A0A0A0MTA6,A0A0A0MTA6_HUMAN,MGNLKSVAQEPGPPCGLGLGLGLGLCGKQGPATPAPEPSRAPASLL...,614,9606


In [352]:
hc_seq = pd.merge(left = merged, right=hc_seq_raw, left_on = 'uniprotID', right_on='Entry', how='inner')

In [433]:
print len(hc_seq)
hc_seq.head(1)

28096


,compID,geneID,entrezID,uniprotID,Entry,Entry name,Sequence,Length,Organism ID
0,119,283,4846,P29474,P29474,NOS3_HUMAN,MGNLKSVAQEPGPPCGLGLGLGLGLCGKQGPATPAPEPSRAPASLL...,1203,9606


In [436]:
sub = hc_seq[['entrezID', 'uniprotID', 'Sequence']].drop_duplicates()

In [60]:
id_len = [len(x) for x in sub.uniprotID]

In [61]:
sub['id_length'] = id_len

In [62]:
sub_b = sub.sort_values(['entrezID', 'id_length']).drop_duplicates(['entrezID'])

In [446]:
print len(sub_b.entrezID.drop_duplicates()), len(sub_b.uniprotID.drop_duplicates()), len(sub_b[~sub_b.uniprotID.isin(tarSeq.Target)].drop_duplicates('uniprotID'))

8182 8155 7590


In [463]:
len(merged[merged.uniprotID.isin(sub_b.uniprotID)].drop_duplicates('uniprotID'))

8155

In [450]:
# gold protein을 제외한 hc만의 protein --> 이후 gold protein 내용도 합쳐야함. 하지만 계산 빨리 하기 위해서 지금은 제외함.
except_gold_protein = sub_b[~sub_b.uniprotID.isin(tarSeq.Target)].drop_duplicates('uniprotID')[['uniprotID', 'Sequence']]

In [466]:
#except_gold_protein.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold.tsv", sep='\t', index=False)
except_gold_protein.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold.tsv", sep='\t', index=False)

In [469]:
tarSeq[['Target', 'Sequence']].drop_duplicates().to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/gold_target.tsv", sep='\t', index=False)

In [331]:
# GO를 위한 compID, geneID, entrezID 파일을 만든다.
hc_go = merged[['compID', 'geneID', 'entrezID']].drop_duplicates()
hc_gp = hc_go[hc_go.entrezID != 'null']

In [332]:
hc_go.head(1)

,compID,geneID,entrezID
0,119,283,4846


In [334]:
print len(hc_go.compID.drop_duplicates())

66


In [341]:
iid = merged.entrezID.drop_duplicates().tolist()
f = open("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/entrez_id.txt", 'w')
for item in iid:
    f.write("%s\r\n" % item)
f.close()

In [342]:
print len(iid)

8325


In [335]:
hc_go.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_gene.tsv", sep='\t', index=False)

In [69]:
tarSim = pd.read_excel("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_gold_indep.xlsx", header=None)

In [70]:
tarSeq = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/drug_target_seq.tsv")

In [37]:
tarSim.head(1)

,0,1,2,3,4,5,6,7,8,9,...,2299,2300,2301,2302,2303,2304,2305,2306,2307,2308
0,1.0,0.007301,0.007301,0.00921,0.00921,0.015303,0.015303,0.015303,0.015303,0.013242,...,0.012339,0.013388,0.005891,0.005685,0.013741,0.011477,0.012288,0.010557,0.015177,0.012273


In [38]:
print tarSim.shape
print tarSeq.shape

(2309, 2309)
(2309, 5)


In [287]:
print len(tarSeq.Target.drop_duplicates())

670


In [299]:
target_sequence = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/protein_seq.txt")

In [309]:
target_sequence.head()

,Entry,Sequence,Length
0,P19113,MMEPEEYRERGREMVDYICQYLSTVRERRVTPDVQPGYLRAQLPES...,662.0
1,Q9UI32,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...,602.0
2,P00488,MSETSRTAFGGRRAVPPNNSNAAEDDLPTVELQGVVPRGVNLQEFL...,732.0
3,P35228,MACPWKFLFKTKFHQYAMNGEKDINNNVEKAPCATSSPVTQDDLQY...,1153.0
4,P37059,MSTFFSDTAWICLAVPTVLCGTVFCKYKKSSGQLWSWMVCLAGLCA...,387.0


In [310]:
#tarSeq[~tarSeq.Target.isin(merged.uniprotID)]
kkk = merged[merged.uniprotID.isin(target_sequence.Entry)]

In [314]:
print len(kkk.uniprotID.drop_duplicates())
print len(merged.uniprotID.drop_duplicates())
print len(target_sequence.Entry.drop_duplicates())

1859
236321
2290


In [316]:
print len(kkk.compID.drop_duplicates())

65


In [39]:
gold_target = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/gold_target.tsv")

In [50]:
herb_target = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold.tsv")

In [14]:
hc_tar_ex = pd.read_excel("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_hc_gold.xlsx", header=None)

In [15]:
hc_tar_ex[hc_tar_ex]

,0,1,2,3,4,5,6,7,8,9,...,7580,7581,7582,7583,7584,7585,7586,7587,7588,7589
0,0.014274,0.009387,0.015436,0.020695,0.020867,0.021093,0.026532,0.017220,0.028348,0.014849,...,0,0,0,0,0,0,0,0,0,0
1,0.013596,0.008355,0.010837,0.011942,0.012551,0.013899,0.012168,0.010569,0.012638,0.011331,...,0,0,0,0,0,0,0,0,0,0
2,0.012501,0.007523,0.011022,0.016099,0.013382,0.012621,0.015997,0.014141,0.016041,0.012395,...,0,0,0,0,0,0,0,0,0,0
3,0.010427,0.008572,0.011361,0.015893,0.014182,0.014131,0.015325,0.012385,0.015291,0.012245,...,0,0,0,0,0,0,0,0,0,0
4,0.019205,0.013697,0.014602,0.023817,0.020186,0.018354,0.015552,0.018959,0.019211,0.015479,...,0,0,0,0,0,0,0,0,0,0


In [45]:
partial = hc_tar_ex[hc_tar_ex.columns.values[:670]]

In [42]:
gold_target.head(1)

,Target,Sequence
0,P35228,MACPWKFLFKTKFHQYAMNGEKDINNNVEKAPCATSSPVTQDDLQY...


In [46]:
partial.columns = gold_target.Target.tolist()

In [52]:
partial.shape

(7590, 670)

In [53]:
partial.index = herb_target.uniprotID.tolist()

In [66]:
partial.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_hc_gold_index.tsv", sep='\t')
partial.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_hc_gold_index.tsv", sep='\t')

In [68]:
except_gold_protein = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_target_except_gold_new.tsv")

In [71]:
hc_tar_ex_new = partial.filter(items = except_gold_protein.uniprotID, axis=0)

In [72]:
gold_indep_target = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/drug_target_seq.tsv")

In [73]:
tarSim.columns = gold_indep_target.Entry.tolist()
tarSim.index = gold_indep_target.Entry.tolist()

In [75]:
print tarSim.shape
print tarSeq.shape
print len(tarSeq.Target.drop_duplicates())

(2309, 2309)
(2309, 5)
670


In [76]:
ext_gold_tar_id = sub_b[sub_b.uniprotID.isin(tarSeq.Target)].drop_duplicates('uniprotID').uniprotID.tolist()

In [77]:
tmp1 = tarSim[~tarSim.index.duplicated()]
tmp2 = tmp1.loc[:, ~tmp1.columns.duplicated()]
ext_gold_tar = tmp2.filter(items = ext_gold_tar_id, axis=0)

In [78]:
hc_tar_sim = pd.concat([hc_tar_ex_new, ext_gold_tar])

In [79]:
hc_tar_sim.head(1)

,P35228,Q9P0X4,P00519,P23219,P21728,P51168,P20309,O43246,Q07869,Q12809,...,Q9UGJ0,Q9UGI9,Q14571,Q14573,Q14123,P51160,P16499,P35913,Q9Y698,Q92806
P11245,0.014274,0.009387,0.015436,0.020695,0.020867,0.021093,0.026532,0.01722,0.028348,0.014849,...,0.034546,0.024622,0.011882,0.010611,0.015694,0.015634,0.0215,0.021294,0.017281,0.022545


In [80]:
gold_tar_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_target_sim_mat.tsv", index_col=0)

In [81]:
hc = hc_seq.compID.tolist()
gd = gold_tar_sim.index.tolist()
df = pd.DataFrame(columns=gd, index=hc)
df.fillna(0.0, inplace=True)

NameError: name 'hc_seq' is not defined